In [65]:
from selenium.webdriver import Chrome
import time
import random
import json
import pandas as pd
import string
import re
import gensim

In [66]:
# browser = Chrome()


# urls = ["https://www.allrecipes.com/recipes/76/appetizers-and-snacks/",
#         "https://www.allrecipes.com/recipes/156/bread/",
#         "https://www.allrecipes.com/recipes/78/breakfast-and-brunch/",
#         "https://www.allrecipes.com/recipes/276/desserts/cakes/",
#         "https://www.allrecipes.com/recipes/201/meat-and-poultry/chicken/",
#         "https://www.allrecipes.com/recipes/362/desserts/cookies/",
#         "https://www.allrecipes.com/recipes/430/seafood/shellfish/shrimp/",
#         "https://www.allrecipes.com/recipes/253/everyday-cooking/slow-cooker/",
#         "https://www.allrecipes.com/recipes/1227/everyday-cooking/vegan/",
#         "https://www.allrecipes.com/recipes/86/world-cuisine/"]

# browser.set_page_load_timeout(300)
# browser.get('https://chrome.google.com/webstore/detail/adblock/gighmmpiobklfepjocnamgkkbiglidom')

In [67]:
# for url in urls:
#     print(url)
#     browser.get(url)

#     for _ in range(6):
#         browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(3 + random.random()*10)

#     link_list = browser.find_elements_by_xpath('//*[contains(@class,"fixed-recipe-card__info")]/a')

#     recipe_page_list = [link.get_attribute('href') for link in link_list]

#     len(recipe_page_list)

#     with open('recipes_list.json', 'a') as f:


#         for recipe in recipe_page_list:

#             browser.get(recipe)

#             recipe_dict = {}
            
#             try:
#                 recipe_dict['title'] = browser.find_element_by_xpath('//*[@id="recipe-main-content"]').text
#             except:
#                 recipe_dict['title'] = browser.find_element_by_xpath('//*[contains(@class, "intro article-info")]/h1').text
            
#             try:
#                 lala_list = browser.find_elements_by_xpath('//*[contains(@id,"lst_ingred")]')
#                 item_list = []
#                 for element in lala_list:

#                     item_list += element.find_elements_by_css_selector('span')

#                 ingredients = [item.text for item in item_list[:-1]]
#                 recipe_dict['ingredients'] = ingredients
            
#             except:
#                 lala_list = browser.find_elements_by_xpath('//*[contains(@class, "ingredients-item-name")]')

#                 ingredients = [item.text for item in lala_list]
#                 recipe_dict['ingredients'] = ingredients

#             json.dump(recipe_dict, f)
#             f.write('\n')
#             time.sleep(3 + random.random()*10)

In [68]:
#browser.find_element_by_xpath('//*[contains(@class, "intro article-info")]/h1').text
# browser.find_element_by_xpath('//*[@id="recipe-main-content"]').text
#browser.find_elements_by_xpath('//*[contains(@class, "ingredients-item-name")]')

In [69]:
#bring in the json data line by line
data = []
with open('recipes_list.json') as file:
    for line in file:
        data.append(json.loads(line))


In [70]:
df = pd.DataFrame(data)

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929 entries, 0 to 928
Data columns (total 2 columns):
ingredients    929 non-null object
title          929 non-null object
dtypes: object(2)
memory usage: 14.6+ KB


In [72]:

# for i in range(928):
#     print(df.ingredients[i], '\n',i,'\n')

In [73]:
indices_list = []
for i in range(len(df.ingredients)):
    
    
        try: 
            if df.ingredients[i][0] == '':
                print(df.ingredients[i])
                indices_list.append(i)
                
        except IndexError:
            print(df.ingredients[i])
            indices_list.append(i)


print(indices_list)

[]
['', '', '', '', '', '', '', '', '', '']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[60, 136, 217, 220, 222, 251, 436, 441, 509, 710, 713, 731, 778, 788, 867]


In [74]:
to_be_reindexed_df = df.drop(indices_list)

In [75]:
to_be_reindexed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 914 entries, 0 to 928
Data columns (total 2 columns):
ingredients    914 non-null object
title          914 non-null object
dtypes: object(2)
memory usage: 21.4+ KB


In [76]:
clean_df = to_be_reindexed_df.reset_index(drop=True)
clean_df.head()

,ingredients,title
0,"[2 pounds unpeeled large shrimp, 2 tablespoons...",Grilled Shrimp Scampi Cocktail
1,"[cooking spray, 24 (1 inch) cubes fresh pineap...",Easy Rumaki with Pineapple
2,"[2 tablespoons butter, 2 tablespoons minced gr...",Perfect Crab-Stuffed Mushrooms
3,"[3 cups diced cooked rotisserie chicken, 2 (8 ...",Baked Buffalo Chicken Dip
4,"[24 slices French baguette, 1 tablespoon butte...",Strawberry Bruschetta


In [77]:
clean_df.ingredients[0]

['2 pounds unpeeled large shrimp',
 '2 tablespoons olive oil',
 '1 teaspoon minced garlic',
 '1/4 teaspoon salt',
 '1/8 teaspoon freshly ground black pepper',
 '1 cup cocktail sauce',
 '1 lemon, cut into wedges']

In [78]:
def split_words_in_str_lst_items(string_list):
    split_words = []
    for item in string_list:
        split_words.append(item.split())
    return split_words
        

def lst_clean_cooking_stop_words(string_list):
    cooking_stop_words = [
    'canned', 'cans', 'drained', 'and', 'halved', 'cup', 'cups',
    'teaspoon', 'tablespoon', 'teaspoons', 'tablespoons',
    'finely', 'freshly', 'fresh', 'thickcut', 'to', 'taste',
    'grated', 'cut', 'into', 'wedges', 'pounds', 'unpeeled', 'large',
    'minced', 'slice', 'slices', 'sliced', 'thick-cut', 'cut', 'crosswise', 'pieces',
    'toothpicks', 'low-fat', 'chopped', 'or', 'taste', 'cooked', 'dry',
    'shredded', 'beaten', 'dried', 'melted', 'stems', 'removed', 'diced',
    'ounce', 'ounces', 'packages', 'softened', 'such', 'RedHot®', 'RedHot', 'Franks', "Frank's",
    'crumbled', 'Old', 'Bay®', 'Bay', 'pinch', 'for', 'garnish', 'slice', 'slices',
    'needed', 'inch', 'cubes', 'cooking', 'spray', 'ground', 'rotisserie',
    'lowfat', 'as', 'quarteres', 'cloves', 'more', 'can', 'package', 'frozen',
    'thawed', 'packet', 'reducedfat', 'Knorr', 'container', 'pound', 'peeled',
    'deveined', 'seeded', 'ripe', 'English', 'juiced', 'plus', 'more', 'Hass',
    'cubed', 'Mexicanstyle', 'hearts', 'prepared', 'party', 'pitted', 'mashed',
    'roma', 'optional', 'chunk', 'Hot', 'bunch', 'cleaned', 'box', 'chickenflavored',
    'Golden', 'delicious', 'cored', 'any', 'flavor', 'flavored', 'whole', 'allpurpose',
    'all', 'purpose', 'deep', 'frying', 'dash', 'packed', 'in', 'French', 'jar',
    'small', 'head', 'little', 'smokie', 'seasoned', 'Boston', 'Bibb', 'leaves',
    'lean', 'pickled', 'Asian', 'dark', 'flaked', 'rolled', 'packed', 'jellied',
    'thirds', 'with', 'attached', 'skewers', 'skinless', 'boneless', 'half', 'kernels',
    'rinsed', 'quart', 'quarts', 'kernel', 'Italianstyle', 'unpopped', 'lightly',
    'coating', 'SAUCE', 'lengthwise', 'miniature', 'semisweet', 'rinsed', 'round',
    'squeezed', 'stewed', 'raw', 'the', 'liquid', 'reserved', 'medium', 'instant',
    'solid', 'pack', 'refrigerated', 'halves', 'distilled', 'loaf', 'extra', 'virgin',
    'crushed', 'kosher', 'toasted', 'buttery', 'TM', 'panko', 'Japanese', 'regular',
    'bottle', 'bottles', 'thin', 'peel', 'paper', 'thick', 'circles', 'unbleached',
    'breast', 'breasts', 'wings', 'strips', 'jumbo', 'giant', 'chunks', 'quickcooking',
    'sweetened', 'flakes', 'Ranchstyle', 'snipped', 'food', 'ROTEL', 'Italian', 'sticks',
    'stick', 'crescent', 'thinly', 'boiled', 'Genoa', 'roasted', 'thin', 'extrasharp',
    'pressed', 'sifted', 'split', 'tips', 'discarded', 'mini', 'deli', 'drain', 'reserve',
    'diameter', 'Greek', 'Thai', 'drops', 'square', 'crusty', 'American', 'selfrising',
    'imitation', 'Wings', 'apart', 'at', 'joints', 'wing', 'tips', 'discarded', 'parts',
    'tops', 'seperated', 'blend', 'coarsely', 'sweet', 'stalk', 'heads', 'husked',
    'divided', 'pats', 'unsalted', 'active', 'warm', 'sea', 'separated', 'herb',
    'overripe', 'degrees', 'F', 'C', 'room', 'temperature', 'machine', 'very',
    'pint', 'puree', 'coarse', 'envelopes', 'lukewarm', 'creamstyle', 'unsweetened',
    'lite', 'of', 'chilled', 'freezer', 'cold', 'brushing', 'nonfat', 'squares',
    'tails', 'thigh', 'quarters'
    


    ]
    cleaned_items = []
    for item_string in string_list:
        for word in item_string:
            if word not in cooking_stop_words:
          #  cleaned_items.append(item_string.replace(item_string, ''))
        #else:
                cleaned_items.append(word)
    return cleaned_items

def lst_strip_numbers_and_punc(string_list):
    num_removed = []
    for x in string_list:
        current_string = re.sub("\d+", "", x)
        current_string = re.sub(r'[^\w\s]','',current_string)
        #current_string = re.sub("", "", current_string)


        num_removed.append(current_string)
    return num_removed

def clean_ingredients(string_list):
    stage_1 = lst_strip_numbers_and_punc(string_list)
    stage_2 = split_words_in_str_lst_items(stage_1)
    stage_3 = lst_clean_cooking_stop_words(stage_2)
    return stage_3
    
    

In [79]:
clean_df.ingredients = clean_df.ingredients.map(clean_ingredients)

In [80]:
model = gensim.models.Word2Vec(clean_df.ingredients, sg=1)

In [81]:
model.train(clean_df.ingredients,
            total_examples=model.corpus_count,
            epochs=model.epochs)

(32070, 69575)

In [82]:
model.corpus_total_words

13915

In [83]:
model.wv

In [84]:
model.wv['flour']

array([ 0.23380232,  0.26510972, -0.10764425, -0.0487986 , -0.03980858,
       -0.10773566, -0.11503976,  0.01225205,  0.2597235 , -0.28346562,
        0.3354803 , -0.01645649, -0.08023246, -0.21793146, -0.04151519,
        0.03927092,  0.29365253,  0.41785192, -0.10500488, -0.00785481,
        0.15141582,  0.13765645,  0.0747885 ,  0.16866873, -0.13691047,
       -0.068515  , -0.00307801,  0.14735734,  0.10173354,  0.07322153,
        0.19946755, -0.09003689,  0.21641031, -0.03251247, -0.11046906,
       -0.1340869 , -0.0497964 , -0.06638785, -0.050085  ,  0.3968453 ,
       -0.25008896,  0.02047445, -0.07258327, -0.08517125,  0.15268332,
        0.23798802,  0.13382107, -0.01594428,  0.16532707, -0.19132984,
        0.15858997,  0.09461148,  0.06422029, -0.07270984,  0.09481903,
       -0.11746266, -0.24120471, -0.05483848, -0.15795541,  0.43156138,
        0.11755814, -0.01265184, -0.15550713,  0.29917467,  0.03186281,
        0.0719915 ,  0.01241397, -0.15411721, -0.21063666, -0.09

In [147]:
parse_ingred_tuples(model.wv.most_similar(positive=['bread', 'butter'],
                                          topn=100000)[-10:])


[('chile', 0.5256317853927612),
 ('corn', 0.522813081741333),
 ('bay', 0.5227915048599243),
 ('tomatoes', 0.5024605989456177),
 ('bell', 0.5002720952033997),
 ('peppers', 0.4962018132209778),
 ('jalapeno', 0.4895881116390228),
 ('kidney', 0.481563925743103),
 ('cumin', 0.47049713134765625),
 ('beans', 0.4573346674442291)]

In [148]:
def parse_ingred_tuples(tupp_list):
    sim_ingreds = []
    for i in range(len(gary)):
        sim_ingreds.append(gary[i][0])
    return sim_ingreds

In [135]:
def ingred_list_compare(ingredient_list, compare_list):
    ing_check_result = []
    for i in ingredient_list:
        if i in compare_list:
            ing_check_result.append(i)
    return ing_check_result

In [137]:
ingredient_check(clean_df.ingredients[0],['shrimp', 'olive', 'oil', 'garlic'])

['shrimp', 'olive', 'oil', 'garlic']

In [129]:
clean_df


,ingredients,title
0,"[shrimp, olive, oil, garlic, salt, black, pepp...",Grilled Shrimp Scampi Cocktail
1,"[pineapple, water, chestnut, bacon, sesameging...",Easy Rumaki with Pineapple
2,"[butter, green, onion, crabmeat, bread, crumbs...",Perfect Crab-Stuffed Mushrooms
3,"[chicken, cream, cheese, hot, pepper, sauce, p...",Baked Buffalo Chicken Dip
4,"[baguette, butter, strawberries, white, sugar]",Strawberry Bruschetta
5,"[onion, quarters, garlic, artichoke, spinach, ...",Amazing No Cook Spinach Artichoke Dip
6,"[clam, juice, oregano, shrimp, tomatoes, cucum...",Mexican-Style Shrimp Cocktail
7,"[artichoke, vegetable, oil, salt, black, peppe...",Artichoke Hearts Gratin
8,"[butter, poppy, seeds, onion, Worcestershire, ...",Easy Ham and Cheese Appetizer Sandwiches
9,"[avocados, lime, salt, onion, cilantro, plum, ...",Guacamole
